**Load library and other requirements**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip3 install torchviz
!pip3 install transformers==3.02

     |████████████████████████████████| 51kB 5.2MB/s 
  Created wheel for torchviz: filename=torchviz-0.0.1-cp36-none-any.whl size=3522 sha256=6c36f829e85021f77a5b0b614d15602cf2ea229e0b7355783e86ee14f5572c1e
  Stored in directory: /root/.cache/pip/wheels/2a/c2/c5/b8b4d0f7992c735f6db5bfa3c5f354cf36502037ca2b585667
Successfully built torchviz
     |████████████████████████████████| 1.0MB 7.3MB/s 
     |████████████████████████████████| 890kB 29.8MB/s 
     |████████████████████████████████| 3.0MB 48.7MB/s 
     |████████████████████████████████| 1.1MB 43.9MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=2538482d8cf460cfb9e2bca8f44b7a346e4ddf3a3349c5e679c7c76d7cc8847e
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [3]:
from tabulate import tabulate
import logging
import os
import argparse
import torch
import numpy as np
from torch.utils.data.dataset import Dataset
from transformers.data.processors.utils import InputExample, InputFeatures
from transformers.data.processors.glue import glue_convert_examples_to_features
from transformers.data.processors.utils import DataProcessor
from transformers import (
    AutoConfig,
    AutoModelForSequenceClassification,
    AutoTokenizer,
    Trainer,
    TrainingArguments,
)

labels = ["-1", "1"]
max_length = 128
logger = logging.getLogger(__name__)


In [4]:
import argparse
import os, sys
import os.path as osp
import numpy as np
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import random, pdb, math, copy
from tqdm import tqdm
from scipy.spatial.distance import cdist
from sklearn.metrics import confusion_matrix
from torch.optim.lr_scheduler import StepLR
import torch.nn.functional as F
import nltk
import re
from nltk.corpus import stopwords
from torch.utils.data import TensorDataset, random_split
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from sklearn.metrics import f1_score,precision_score,recall_score,confusion_matrix
import math
import copy 
import time
from torch.autograd import Variable

In [5]:
# Set the seed value all over the place to make this reproducible.

def seed_everything(seed=3000):
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    import os
    os.environ['PYTHONHASHSEED'] = str(seed)

seed_everything()
train_min_step = 1000
train_lr=0.0001
train_weight_decay = 0.0005
train_momentum = 0.9
train_update_freq = 50
# for step, batch in enumerate(train_dataloader):
#   print(batch)

In [6]:
import torch
from torch.autograd import Variable
from torch.nn.utils.rnn import pad_sequence
model_name = "tmills/roberta_sfda_sharpseed"

config = AutoConfig.from_pretrained(model_name,output_hidden_states=True)
tokenizer = AutoTokenizer.from_pretrained(model_name,
                                          config=config)

fixed_source_net = AutoModelForSequenceClassification.from_pretrained(model_name,
                                                            config=config)
fixed_source_net.cuda()
for k,v in fixed_source_net.named_parameters():
  v.requires_grad = False

In [7]:
class NegationDataset(Dataset):
    def __init__(self, features):
        self.features = features
        self.label_list = ["-1", "1"]

    def __len__(self):
        return len(self.features)

    def __getitem__(self, i) -> InputFeatures:
        return self.features[i]

    def get_labels(self):
        return self.label_list


    @classmethod
    def from_tsv(cls, tsv_file, tokenizer,flag=0,idx = [],y_true_ = [],extra_ratio=0):
        """Creates examples for the test set."""
        lines = DataProcessor._read_tsv(tsv_file)
        # print(lines)
        pos_lines = []
        neg_lines = []
        extra_lines  = 0
        pos_n = 0
        neg_n = 0
        if idx!=[]:
          lines = [line for (i,line) in enumerate(lines) if i in idx]
          pos_lines = [line for (i,line) in enumerate(lines) if y_true_[i]==1]
          neg_lines = [line for (i,line) in enumerate(lines) if y_true_[i]==0]
          pos_n = len(pos_lines)
          neg_n = len(neg_lines)
          extra_pos_ratio = ((len(neg_lines)/len(pos_lines))*(1+extra_ratio)-1)
          extra_neg_ratio = extra_ratio
          # print(extra_pos_ratio)

        # print(lines)
        if flag==1:
          pos_y_true_ = torch.ones(pos_n).cuda()
          neg_y_true_ = torch.zeros(neg_n).cuda()
          pos_lines,pos_y_true_ = text_augmentation(pos_lines,pos_y_true_,extra_pos_ratio)
          neg_lines,_ = text_augmentation(neg_lines,neg_y_true_,extra_neg_ratio)
          lines = []
          # print(len(pos_y_true_))
          y_true_ = pos_y_true_.unsqueeze(1)
          for line in pos_lines:
            lines.append(line)
          for line in neg_lines:
            lines.append(line)
            y_true_ = torch.vstack((y_true_,torch.tensor(0).cuda()))
          
          y_true_ = y_true_.squeeze(1)
            
        examples = []
        for (i, line) in enumerate(lines):
            # if idx==None or i in idx:
            guid = 'instance-%d' % i
            if line[0] in labels:
                text_a = '\t'.join(line[1:])
            else:
                text_a = '\t'.join(line)
            examples.append(InputExample(guid=guid, text_a=text_a, text_b=None, label=None))

        features = glue_convert_examples_to_features(
            examples,
            tokenizer,
            max_length=max_length,
            label_list=labels,
            output_mode='classification',
        )
        return cls(features),y_true_

**Training Code**

In [ ]:

## text augmentation code
def text_augmentation(lines,y_true_,extra_size_ratio=5):
  lt = 6
  rt = 3
  y_true_= y_true_.unsqueeze(1)
  p = re.compile('<e>.*</e>')
  concept_terms = []
  for line in lines:
    line=line[0]
    concept_terms.append(re.findall('<e>.*</e>', line)[0])
  sz = len(lines)
  for i in range(int(extra_size_ratio*sz)):
    idx = random.choice(range(sz))
    line = lines[idx][0]
    n=len(line)
    ## taking only a part of sentence that contains the marked term with max 6 words on left and 3 word on right
    m=p.search(line)
    (start,end) = m.span()

    ## replacing concept term with some other concept term
    idx2 = random.choice(range(len(concept_terms)))
    new_line2 = line[:start] + concept_terms[idx2] + line[end:]
    lines.append([new_line2])
    # lines.append([line])
    # print(torch.tensor([y_true_[idx].cpu().numpy()]))
    y_true_ = torch.vstack((y_true_,torch.tensor(y_true_[idx].cpu().numpy()).cuda()))
    ## -------
    # print("original line : ",line)
    # print("new first line : ",new_line)
    # print("new second line : ",new_line2)
  y_true_ = y_true_.squeeze(1)
  return lines,y_true_


train_dataset,_ = NegationDataset.from_tsv('drive/My Drive/source-free-domain-adaptation/practice_text/negation/train.tsv', tokenizer)

In [ ]:


## preprocesses inputs from the text that can be directly fed to model

input_ids = []
attention_masks = []
output_label = []
y_true = torch.tensor([]).cuda()
entropyl = torch.tensor([]).cuda()
m=torch.nn.Softmax(dim=1)
cnt=0
for feat in train_dataset.features:
  input_ids.append(feat.input_ids)
  attention_masks.append(feat.attention_mask)
  out = fixed_source_net(torch.tensor([feat.input_ids]).cuda(),torch.tensor([feat.attention_mask]).cuda())

  pred = m(out[0])
  # print(pred)
  entropy = torch.sum(- pred * torch.log(pred), dim=1, keepdim=True)
  entropy_norm = entropy / np.log(pred.size(1))
  entropy_norm = entropy_norm.squeeze(1)
  # print(entropy_norm)
  entropyl = torch.cat((entropyl,entropy_norm),0)
  z = torch.argmax(out[0],dim=1)
  z = z.type(torch.cuda.LongTensor)
  y_true = torch.cat((y_true,z),0)
  # print(y_true)

print(entropyl)
input_ids = torch.tensor(input_ids)
attention_masks = torch.tensor(attention_masks)

tensor([0.0259, 0.0041, 0.0039,  ..., 0.0041, 0.1138, 0.0042], device='cuda:0')


In [ ]:

pos_entropyl = torch.sort(entropyl[y_true==1])
neg_entropyl = torch.sort(entropyl[y_true==0])
pos_threshold = torch.quantile(pos_entropyl.values,torch.tensor([0.5]).cuda())
neg_threshold = torch.quantile(neg_entropyl.values,torch.tensor([0.5]).cuda())


print(pos_threshold)
pos_mask = (y_true==1) & (entropyl<=pos_threshold)
print(len(input_ids[pos_mask]))
neg_mask = (y_true==0) & (entropyl<pos_threshold)
print(len(input_ids[neg_mask]))


mask = (((entropyl<pos_threshold) & (y_true==1)) | ((entropyl<pos_threshold) & (y_true==0)))
idx = torch.where(mask==True)[0]
# new_train_dataset = NegationDataset.from_tsv('drive/My Drive/source-free-domain-adaptation/practice_text/negation/train.tsv', tokenizer,1,idx.tolist())
y_true_ = y_true[idx]
new_train_dataset,y_true_= NegationDataset.from_tsv('drive/My Drive/source-free-domain-adaptation/practice_text/negation/train.tsv', tokenizer,1,idx.tolist(),y_true_,3)

print(len(y_true_[y_true_==0]))
print(len(y_true_[y_true_==1]))
print(len(new_train_dataset.features))

tensor([0.1129], device='cuda:0')
333
2137
8548
8548
17096


In [ ]:
new_input_ids = []
new_attention_masks = []

for feat in new_train_dataset.features:
  new_input_ids.append(feat.input_ids)
  new_attention_masks.append(feat.attention_mask)

new_input_ids = torch.tensor(new_input_ids)
new_attention_masks = torch.tensor(new_attention_masks)
y_true_ = torch.tensor(y_true_)
y_true_ = y_true_.type(torch.cuda.LongTensor)



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # Remove the CWD from sys.path while we load stuff.


In [ ]:
dataset = TensorDataset(new_input_ids, new_attention_masks,y_true_)
batch_size=16 # 32
train_dataloader = DataLoader(
            dataset,  # The training samples.
            # sampler = RandomSampler(dataset), # Select batches randomly        ### was commented for 0.8759
            shuffle = True,  ### not here for 0.8759
            batch_size = batch_size # Trains with this batch size.
        )
print(len(dataset))

17096


In [ ]:

def op_copy(optimizer):
    for param_group in optimizer.param_groups:
        param_group['lr0'] = param_group['lr']
    return optimizer

def lr_scheduler(optimizer, iter_num, max_iter, gamma=10, power=0.75):
    decay = (1 + gamma * iter_num / max_iter) ** (-power)
    # decay = 1/math.sqrt(iter_num)
    for param_group in optimizer.param_groups:
        param_group['lr'] = param_group['lr0'] * decay
        param_group['weight_decay'] =0.0005
        param_group['momentum'] = 0.9
        param_group['nesterov'] = True
    return optimizer

def tensor_l2normalization(q):
    qn = torch.norm(q, p=2, dim=1).detach().unsqueeze(1)
    q = q.div(qn.expand_as(q))
    return q

def forward(y_logits, y_true):
    beta = 1
    epsilon = 1e-15
    # print(y_logits)
    y_pred = torch.sigmoid(y_logits)
    # m=torch.nn.Softmax(dim=1)
    # y_pred = m(y_logits)
    # print(y_pred)
    
    TP = (y_pred * y_true).sum(dim=1)
    FP = (y_pred * (1-y_true)).sum(dim=1)
    FN = ((1-y_pred) *  y_true).sum(dim=1)
    fbeta = (1 + beta**2) * TP / ((1 + beta**2) * TP + (beta**2) * FN + FP + epsilon)
    fbeta = fbeta.clamp(min=epsilon, max=1 - epsilon)
    return 1 - fbeta.mean()


In [ ]:
model_name = "tmills/roberta_sfda_sharpseed"
# del config1,tokenizer1,target_trainable_net
config1 = AutoConfig.from_pretrained(model_name,output_hidden_states=True)
tokenizer1 = AutoTokenizer.from_pretrained(model_name,
                                          config=config1)

target_trainable_net = AutoModelForSequenceClassification.from_pretrained(model_name,
                                                            config=config1)
target_trainable_net = target_trainable_net.cuda()
class classifier_head(nn.Module):
  def __init__(self):
      super().__init__()
      # self.dense = nn.Linear(in_features = 768,out_features = 768,bias=True)
      self.dense = copy.deepcopy(target_trainable_net.classifier.dense)
      # self.dropout = nn.Dropout(p=0.1, inplace = False)
      self.dropout = copy.deepcopy(target_trainable_net.classifier.dropout)
      # self.out_proj = nn.Linear(in_features = 768, out_features = 2,bias=True)
      self.out_proj = copy.deepcopy(target_trainable_net.classifier.out_proj)
  def forward(self, x):
      # take <s> token (equiv. to [CLS])
      # x = feature[:,0,:]
      x = self.dropout(x)
      x = self.dense(x)
      x = torch.tanh(x)
      x = self.dropout(x)
      x = self.out_proj(x)
      return x

classifier = classifier_head()
classifier = classifier.cuda() ### change

In [ ]:
## training portion of the code
l=[]
for itr in range(1):
  del target_trainable_net,classifier
  target_trainable_net = AutoModelForSequenceClassification.from_pretrained(model_name,config=config1)
  target_trainable_net = target_trainable_net.cuda()
  classifier = classifier_head()
  classifier = classifier.cuda()
  for k,v in target_trainable_net.named_parameters():
    x = k.split('.')
    if x[0]=='classifier':
      v.requires_grad=False


  train_start_time=time.time()

  train_min_step = 3204#2650#1335#2670#930#188*3 ### 1000 originally
  train_lr=0.0005
  train_weight_decay = 0.0005
  train_momentum = 0.9
  train_update_freq = 96 #96 # changed from 10

  # target_trainable_net.train()
  optimizer = optim.SGD(list(filter(lambda p: p.requires_grad, target_trainable_net.parameters()))+list(classifier.parameters()), 
                        lr=train_lr, weight_decay=train_weight_decay, momentum=train_momentum, nesterov=True)

  optimizer = op_copy(optimizer)
  global_step = 0
  best_acc = 0
  best_epoch = 0
  epoch_id = 0
  class_num =  2
  max_epoch = math.ceil((train_min_step*batch_size)/len(dataset))
  pt_memory_update_frequncy =  train_update_freq
  # eval_interval = 100
  cnt=0
  m=torch.nn.Softmax(dim=1)

  # fixed_source_net.train()
  classifier.train()

  # fixed_source_net.zero_grad()
  classifier.zero_grad()
  target_trainable_net.zero_grad()

  while global_step < train_min_step:
    epoch_id += 1
    max_iter = max_epoch * len(train_dataloader)
    # print(max_iter)
    epoch_start_time=time.time()

    for i, dt in enumerate(train_dataloader):
      # APM init/update
      # if (global_step) % pt_memory_update_frequncy == 0: ### change
      #     target_trainable_net.eval() ### change
      #     prototype_memory, num_prototype_,prototype_memory_dict = APM_ours(target_trainable_net)
          # prototype_memory,prototype_memory_dict,num_prototype_ = proto_augmentation(prototype_memory,prototype_memory_dict,num_prototype_) # Tried augmentation in prototype feature

      input_id = dt[0].cuda()
      attention_mask = dt[1].cuda()
      pseudo_label = dt[2].cuda()
      fixed_source_net.train()
      out_s = fixed_source_net(input_id,attention_mask=attention_mask)
      pseudo_label_s = torch.argmax(out_s[0], dim=1).cuda()
      pseudo_label_hot_s = torch.zeros(out_s[0].shape).cuda()
      pseudo_label_hot_s = pseudo_label_hot_s.scatter(1,pseudo_label_s.unsqueeze(1),1.0).cuda()
      ## trainable target model
      target_trainable_net.train()
      out_t = target_trainable_net(input_id,attention_mask=attention_mask)
      fc_t = out_t[1][-1][:,0,:] #target_trainable_net.roberta.pooler(out_t[1][-1])
      feature_embed_tensor = fc_t.cpu()
      
      
      logit_s2t = classifier(fc_t) ### change
      logit_t = out_t[0]
      
      ce_from_s2t = nn.CrossEntropyLoss()(logit_t,pseudo_label)
      ce_total = ce_from_s2t
      if global_step%2==0:
        batch_loss = ce_total
      else:
        batch_loss+=ce_total
      global_step+=1 
      if global_step%2==0:
        # print(optimizer.param_groups[0]['lr'])
        optimizer.zero_grad()
        batch_loss /= 2
        batch_loss.backward(retain_graph=True)
        # print(global_step,batch_loss)
        optimizer.step()
        lr_scheduler(optimizer, iter_num=global_step, max_iter=max_iter)


    epoch_end_time=time.time()
    print("time taken for epoch no. {} is {}".format(epoch_id,epoch_end_time-epoch_start_time))
    target_trainable_net.eval()
    fixed_source_net.eval()
    # fscore,prec,recall = evaluation(test_dataloader,target_trainable_net,fixed_source_net)
    # l.append((fscore,prec,recall))
    # print(l)
  print(cnt)
  train_end_time = time.time()
  print("Total_time taken : ",train_end_time-train_start_time)

# print(avg_acc)

time taken for epoch no. 1 is 526.8594431877136
time taken for epoch no. 2 is 526.2461619377136
time taken for epoch no. 3 is 526.1528825759888
0
Total_time taken :  1579.2682042121887


**Predict Labels**

In [8]:
batch_size=16
def load_testdata(file_name):
    test_dataset,_ = NegationDataset.from_tsv(file_name, tokenizer)

    input_ids = []
    attention_masks = []
    for feat in test_dataset.features:
        input_ids.append(feat.input_ids)
        attention_masks.append(feat.attention_mask)

    input_ids = torch.tensor(input_ids)
    attention_masks = torch.tensor(attention_masks)
    print(input_ids.shape,attention_masks.shape)

    dataset = TensorDataset(input_ids, attention_masks,torch.arange(input_ids.size(0)))
    
    # batch_size=32 # 32
    test_dataloader = DataLoader(dataset, batch_size = batch_size)
    print(batch_size)
    return test_dataloader

In [9]:
tokenizer = AutoTokenizer.from_pretrained('drive/My Drive/SFDA/Negation/proto_aug')
target_trainable_net = AutoModelForSequenceClassification.from_pretrained('drive/My Drive/SFDA/Negation/proto_aug')
target_trainable_net.cuda()
def predict(test_file,output_test_file,threshold = 0.5):
  # target_trainable_net = 
  # tokenizer =
  start=True
  test_dataloader = load_testdata(test_file)
  for data in test_dataloader:
    # out = fixed_source_net(data[0].cuda(),data[1].cuda())
    out = target_trainable_net(data[0].cuda(),data[1].cuda())
    if start:
        predict = out[0].cpu().detach().numpy()
        start = False
    else:
        predict = np.concatenate((predict,out[0].cpu().detach().numpy()))
  m=torch.nn.Softmax(dim=1)
  p = m(torch.tensor(predict))
  p1 = np.ones(p.shape[0],np.int)
  for i in range(p.shape[0]):
    prob = p[i][1]
    if prob>threshold:
      p1[i] = 1
    else:
      p1[i] = -1
  # predict = np.argmax(predict,1)
  with open(output_test_file, "w") as writer:
    logger.info("***** Test results *****")
    for index, item in enumerate(p1):
        writer.write("%s\n" % p1[index])

  return p1

In [ ]:
# predict('path_to_test_file','path_where_to_output_predicted_labels')
# predict('drive/My Drive/SFDA/Negation/task1_test.tsv','predicted_labels1.tsv')

**Evaluate Performance Metrics**

In [11]:
def read_tsv(file):
    output = []
    with open(file, 'r') as f_output:
        for record in f_output:
            output.append(int(record))
    return output

def score_negation(ref_domain,res_domain):
    ref = read_tsv(ref_domain)
    res = read_tsv(res_domain)
    assert len(ref) == len(res)
    trainable_f1_score = f1_score(ref,res)
    trainable_prec = precision_score(ref,res)
    trainable_recall = recall_score(ref,res)
    scores = [['trained',trainable_f1_score,trainable_prec,trainable_recall]]
    print(tabulate(scores,headers=['model','f1 score','precision','recall']))
    return trainable_f1_score,trainable_prec,trainable_recall



In [ ]:
# score_negation('path_to_true_labels','path_to_predicted_labels')